In [1]:
import numpy as np
import os
from pydrake.common import temp_directory, FindResourceOrThrow
from pydrake.geometry import (
    MeshcatVisualizer, MeshcatVisualizerParams, 
    Role, StartMeshcat
)

from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import ModelVisualizer

from manipulation.scenarios import AddMultibodyTriad, AddPackagePaths

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7003


# MODEL VIEWING

In [3]:
iiwa7_model_url = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")
iiwa7_model_url

'/home/dema/PycharmProjects/robot_manipulator/lib/python3.10/site-packages/pydrake/common/../lib/../share/drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf'

In [4]:
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModels(file_name=iiwa7_model_url)

[ModelInstanceIndex(2)]

In [5]:
# When this notebook is run in test mode it needs to stop execution without
# user interaction. For interactive model visualization you won't normally
# need the 'loop_once' flag.
test_mode = True if "TEST_SRCDIR" in os.environ else False

# Start the interactive visualizer.
# Click the "Stop Running" button in MeshCat when you're finished.
visualizer.Run(loop_once=test_mode)



Click 'Stop Running' or press Esc to quit


<RunResult.STOPPED: 2>

In [6]:
schunk_wsg50_model_url = FindResourceOrThrow(
    "drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf"
)

In [7]:
schunk_wsg50_model_url

'/home/dema/PycharmProjects/robot_manipulator/lib/python3.10/site-packages/pydrake/common/../lib/../share/drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf'

In [8]:
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModels(file_name=schunk_wsg50_model_url)
visualizer.Run(loop_once=False)

Click 'Stop Running' or press Esc to quit


<RunResult.STOPPED: 2>

# Model DEF sdf

In [9]:
cylinder_sdf = """
<?xml version="1.0"?>
<sdf version="1.7">
    <model name="cylinder">
        <pose>0 0 0 0 0 0</pose>
        <link name="cylinder_link">
            <inertial>
                <mass>1.0</mass>
                <inertia>
                  <ixx>0.005833</ixx>
                  <ixy>0.0</ixy>
                  <ixz>0.0</ixz>
                  <iyy>0.005833</iyy>
                  <iyz>0.0</iyz>
                  <izz>0.005</izz>
                </inertia>
            </inertial>

            <collision name="collision">
                <geometry>
                    <cylinder>
                        <radius>0.1</radius>
                        <length>0.2</length>
                    </cylinder>
                </geometry>
            </collision>

            <visual name="visual">
                <geometry>
                    <cylinder>
                        <radius>0.1</radius>
                        <length>0.2</length>                    
                    </cylinder>
                </geometry>
                <material>
                    <diffuse>1.0 1.0 1.0 1.0</diffuse>
                </material>
            </visual>
        </link>
    </model>
</sdf>
"""

visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelsFromString(cylinder_sdf, 'sdf')
visualizer.Run(loop_once=True)

<RunResult.LOOP_ONCE: 1>

In [10]:
path = '../../robot_models/Planar-v2.sdf'
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelFromFile(path)
visualizer.Run(loop_once=False)

Click 'Stop Running' or press Esc to quit


<RunResult.STOPPED: 2>

In [11]:
full_2r_directives = '''
directives: 
- add_model: 
    name: 2R_planar_robot
    file: file:///home/dema/PycharmProjects/robot_manipulator/robot_models/Planar-v2.sdf
    default_joint_positions: 
        q1: [0]
        q2: [0]

- add_model: 
    name: end_effector
    file: file:///home/dema/PycharmProjects/robot_manipulator/lib/python3.10/site-packages/pydrake/common/../lib/../share/drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld: 
    parent: 2R_planar_robot::l2
    child: end_effector::body_frame
    X_PC: 
        translation: [0.5, 0, 0]
        rotation: !Rpy {deg: [0, 0, -90]}
'''
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelsFromString(full_2r_directives, '.dmd.yaml')
visualizer.Run(loop_once=False)


Click 'Stop Running' or press Esc to quit


<RunResult.STOPPED: 2>

In [11]:
import urllib.parse

def path_to_uri(path):
    """Convert a local file path to a file URI."""
    # Use the 'file://' scheme for local file URIs
    # Encode the path to handle spaces and other special characters
    return urllib.parse.urljoin('file:', urllib.parse.quote(path))

In [12]:
path_to_uri('../robot_models/planar_2R.sdf')

'file:///robot_models/planar_2R.sdf'

# MultiOBJ Scene (robots/objects)

In [13]:
# original code save all in a temp_dir, i'd like to store all: './model'
table_top_sdf_file = './model/table_top.sdf'
table_top_sdf = """
<?xml version="1.0"?>
<sdf version="1.7">
    <model name="table_top">
        <link name="table_top_link">
            
            <visual name="visual">
                <pose>0 0 0.445 0 0 0</pose>
                <geometry>
                    <box>
                        <size>0.55 1.1 0.05</size>
                    </box>
                </geometry>
                <material>
                    <diffuse>0.9 0.8 0.7 1.0</diffuse>
                </material>
            </visual>
            
            <collision name="collision">
                <pose>0 0 0.445 0 0 0</pose>
                <geometry>
                    <box>
                        <size>0.55 1.1 0.05</size>
                    </box>
                </geometry>
            </collision>
            
        </link>
        
        <frame name="table_top_center">
            <pose relative_to="table_top_link">0 0 0.47 0 0 0</pose>
        </frame>
    </model>
</sdf>
"""

with open(table_top_sdf_file, 'w') as f:
    f.write(table_top_sdf)
    


In [14]:
cracker_box = FindResourceOrThrow('drake/manipulation/models/ycb/sdf/003_cracker_box.sdf')

def create_scene(sim_time_step=0.0001): 
    meshcat.Delete()
    meshcat.DeleteAddedControls()
    
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(
        builder, time_step=sim_time_step
    )
    parser = Parser(plant)
    parser.AddModelFromFile('../robot_models/planar_2R.sdf')
    parser.AddModelsFromString(cylinder_sdf, 'sdf')
    parser.AddModels(table_top_sdf_file)
    
    
    table_frame = plant.GetFrameByName('table_top_center')
    plant.WeldFrames(plant.world_frame(), table_frame)
    
    
    plant.Finalize()
    plant_context = plant.CreateDefaultContext()
    
    # connect actuation port
    torque_port = plant.get_actuation_input_port()
    
    # set initial pose for all the free body
    # table, cylinder, arm
    cylinder = plant.GetBodyByName('cylinder_link')
    X_WorldTable = table_frame.CalcPoseInWorld(plant_context)
    X_TableCylinder = RigidTransform(
        RollPitchYaw(np.asarray([90, 0, 0]) * np.pi/180), 
        p=[0, 0, 0.5]
    )
    X_WorldCylinder = X_WorldTable.multiply(X_TableCylinder)
    plant.SetDefaultFreeBodyPose(cylinder, X_WorldCylinder)
    
    Manipulator = plant.GetBodyByName('l1')
    X_TableManipulator= RigidTransform(
        RollPitchYaw(np.asarray([45, 30, 0]) * np.pi / 180),
        p=[0, 0, 0.1]
    )
    X_WorldManipulator = X_WorldTable.multiply(X_TableManipulator)
    plant.SetDefaultFreeBodyPose(Manipulator, X_WorldManipulator)
    
    
    # visualizer 
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat, 
        MeshcatVisualizerParams(role=Role.kPerception, prefix="visual")
    )
    diagram = builder.Build()
    return diagram, visualizer 


def init_simulation(diagram): 
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1.)
    return simulator

def run_simulation(sim_time_step=0.0001): 
    diagram, visualizer = create_scene(sim_time_step)
    simulator = init_simulation(diagram)
    visualizer.StartRecording()
    simulator.AdvanceTo(5.0)
    visualizer.PublishRecording()
    

run_simulation()

ERROR:drake:SDFormat Error [parser.cc:735] Error finding file [/home/dema/PycharmProjects/robot_manipulator/drake_kit/from_tutorial/../robot_models/planar_2R.sdf].


RuntimeError: error: Unable to read file:/home/dema/PycharmProjects/robot_manipulator/drake_kit/from_tutorial/../robot_models/planar_2R.sdf

In [ ]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(
    builder, time_step=0.1
)
parser = Parser(plant)
parser.AddModelFromFile('../robot_models/planar_2R.sdf')
plant.Finalize()
torque_port = plant.get_actuation_input_port()
